In [116]:
# 모듈 임포트
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import joblib
import numpy as np
import psutil
import pandas as pd
import csv
from datetime import datetime
import tflite_runtime.interpreter as tflite
from sklearn.preprocessing import StandardScaler

In [117]:
# TFLite 모델 및 스케일러 로드
interpreter = tflite.Interpreter(model_path="./model/cnn1d_model_compatible_new.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

scaler = joblib.load('./model/scaler_cnn_new.joblib')

In [118]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c)
ads.gain = 2/3

voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

In [119]:
# 센서 보정값
ZMPT_offset = 2.5
ZMPT_scale = 761.8

ACS712_offset = 2.5
ACS712_sensitivity = 0.04598

In [120]:
# 파생 피처 추출 함수
def extract_features_from_sequence(seq):
    voltage_seq, current_seq = zip(*seq)
    voltage = np.mean(voltage_seq)
    current = np.mean(current_seq)
    voltage_diff = np.abs(voltage_seq[-1] - voltage_seq[-2])
    current_diff = np.abs(current_seq[-1] - current_seq[-2])
    voltage_ma = np.mean(voltage_seq[-5:])
    current_ma = np.mean(current_seq[-5:])
    power = voltage * current
    power_diff = np.abs(power - (voltage_seq[-2] * current_seq[-2]))
    return [voltage, current, voltage_diff, current_diff, voltage_ma, current_ma, power, power_diff]

In [121]:
# CNN 입력 시퀀스 생성
def build_cnn_input(data_buffer, seq_len=8):
    if len(data_buffer) < seq_len + 7:
        return None

    sequence = []
    # 마지막 8개의 연속 시퀀스를 뽑기 위한 시작점 조정
    for i in range(len(data_buffer) - (seq_len + 7), len(data_buffer) - 7):
        window = data_buffer[i:i+8]
        if len(window) < 8:
            continue  # 예외 방지
        voltage_seq, current_seq = zip(*window)
        features = extract_features_from_sequence(list(zip(voltage_seq, current_seq)))
        sequence.append(features)

    if not sequence:
        return None
    return np.array(sequence)


In [122]:
# 후처리: 연속 이상 감지 확인
def postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=8):
    if len(anomaly_buffer) < min_consecutive:
        return False
    count = 0
    for val in reversed(anomaly_buffer):
        if val == 1:
            count += 1
        else:
            break
    return count >= min_consecutive

In [126]:
# 테스트 데이터 불러오기 (센서 대체용 - 사용 안 할 경우 생략 가능)
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current']
df = pd.read_csv('./log/raw_dataset_arc.csv', names=column_names, header=None)
df = df.dropna(subset=['voltage', 'current']).reset_index(drop=True)
csv_filename = "./log/0529/학습데이터/노트북 정상 데이터.csv"

In [127]:
try:
    print("🚀 실시간 아크 감지 시작합니다!")

    USE_SENSOR = True  # ← False 설정 시 CSV 모드로 작동
    ANOMALY_THRESHOLD = 2

    data_buffer = []
    anomaly_buffer = []
    first_anomaly_detected = False
    anomaly_start_time = None
    start_time = time.perf_counter()
    idx = -1
    anomaly = 0
    while True:
        idx += 1

        # 센서 or CSV 입력
        if USE_SENSOR:
            try:
                v_raw_sensor = voltage_ch.voltage
                c_raw_sensor = current_ch.voltage
                voltage = (v_raw_sensor - ZMPT_offset) * ZMPT_scale
                current = (c_raw_sensor - ACS712_offset) / ACS712_sensitivity
                prev_voltage = voltage
                prev_current = current
            except (OSError, ValueError, RuntimeError) as e:
                print(f"⚠️ 센서 오류 발생: {e}")
                voltage = prev_voltage if 'prev_voltage' in locals() else 0
                current = prev_current if 'prev_current' in locals() else 0
        else:
            if idx >= len(df):
                break
            voltage = df['voltage'].iloc[idx]
            current = df['current'].iloc[idx]

        data_buffer.append((voltage, current))
        if len(data_buffer) > 100:
            data_buffer.pop(0)

        # CNN 입력 생성 및 추론
        if len(data_buffer) >= 15:
            X_seq = build_cnn_input(data_buffer)
            if X_seq is None:
                continue
            X_scaled = scaler.transform(X_seq)
            X_scaled = X_scaled.reshape(1, 8, 8)

            interpreter.set_tensor(input_details[0]['index'], X_scaled.astype(np.float32))
            interpreter.invoke()
            output_data = interpreter.get_tensor(output_details[0]['index'])
            y_pred = output_data[0][0]
            anomaly = int(y_pred > 0.4) # 예측 퍼센트

            anomaly_buffer.append(anomaly)
            if len(anomaly_buffer) > ANOMALY_THRESHOLD:
                anomaly_buffer.pop(0)

            if anomaly == 1 and anomaly_start_time is None:
                anomaly_start_time = time.perf_counter()
            elif anomaly == 0:
                anomaly_start_time = None

            if postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=ANOMALY_THRESHOLD):
                print("⚡ 아크 이상 감지!", idx)
                if not first_anomaly_detected and anomaly_start_time is not None:
                    total_elapsed = time.perf_counter() - start_time
                    actual_elapsed = time.perf_counter() - anomaly_start_time
                    print(f"⏱️ 전체 소요 시간: {total_elapsed:.6f}초")
                    print(f"⏱️ 연속 {ANOMALY_THRESHOLD}개 이상 신호까지 소요 시간: {actual_elapsed:.6f}초")
                    first_anomaly_detected = True

                    # CSV 기록
                    with open(csv_filename, mode='a', newline='') as file:
                        writer = csv.writer(file)
                        timestamp = datetime.now().isoformat()
                        writer.writerow([timestamp, v_raw_sensor, c_raw_sensor, voltage, current, anomaly])
                        
                    # 상태 출력
                    cpu_usage = psutil.cpu_percent(interval=0.0)
                    memory_usage = psutil.Process().memory_info().rss / 1024 ** 2
                    print(f"🧠 메모리: {memory_usage:.2f}MB | 🧮 CPU: {cpu_usage:.2f}% | idx: {idx}")
                    print(f"v_raw:{voltage:.4f}, current_raw:{current:.4f}, anomaly:{0}")
                    # break

        # CSV 기록
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            timestamp = datetime.now().isoformat()
            writer.writerow([timestamp, v_raw_sensor, c_raw_sensor, voltage, current, 0])

        # 상태 출력
        cpu_usage = psutil.cpu_percent(interval=0.0)
        memory_usage = psutil.Process().memory_info().rss / 1024 ** 2
        print(f"🧠 메모리: {memory_usage:.2f}MB | 🧮 CPU: {cpu_usage:.2f}% | idx: {idx}")
        print(f"v_raw:{v_raw_sensor:.4f}, current_raw:{c_raw_sensor:.4f}, anomaly:{anomaly}")

        time.sleep(0.001163)  # 860Hz 기준

except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")

🚀 실시간 아크 감지 시작합니다!
🧠 메모리: 178.34MB | 🧮 CPU: 3.30% | idx: 0
v_raw:2.4551, current_raw:2.5149, anomaly:0
🧠 메모리: 178.34MB | 🧮 CPU: 20.00% | idx: 1
v_raw:2.2071, current_raw:2.5162, anomaly:0
🧠 메모리: 178.34MB | 🧮 CPU: 0.00% | idx: 2
v_raw:2.7103, current_raw:2.5166, anomaly:0
🧠 메모리: 178.34MB | 🧮 CPU: 0.00% | idx: 3
v_raw:2.8847, current_raw:2.5151, anomaly:0
🧠 메모리: 178.34MB | 🧮 CPU: 0.00% | idx: 4
v_raw:2.3539, current_raw:2.5134, anomaly:0
🧠 메모리: 178.34MB | 🧮 CPU: 11.10% | idx: 5
v_raw:2.2395, current_raw:2.5159, anomaly:0
🧠 메모리: 178.34MB | 🧮 CPU: 0.00% | idx: 6
v_raw:2.7746, current_raw:2.5159, anomaly:0
🧠 메모리: 178.34MB | 🧮 CPU: 0.00% | idx: 7
v_raw:2.8009, current_raw:2.5142, anomaly:0
🧠 메모리: 178.34MB | 🧮 CPU: 0.00% | idx: 8
v_raw:2.2723, current_raw:2.5151, anomaly:0
🧠 메모리: 178.34MB | 🧮 CPU: 12.50% | idx: 9
v_raw:2.2811, current_raw:2.5161, anomaly:0
🧠 메모리: 178.34MB | 🧮 CPU: 0.00% | idx: 10
v_raw:2.8193, current_raw:2.5162, anomaly:0
🧠 메모리: 178.34MB | 🧮 CPU: 22.20% | idx: 11
v_raw:2.732